<a href="https://colab.research.google.com/github/MarkzDG/Metodos_estadisticos/blob/main/Stepwise/Stepwise_foward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este ejemplo se quiere aplicar la metodología de Stepwise foward para encontar el mejor ajuste para encontrar una ecuación para el rendimiento de la soja en función de las demás variables

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

##Carga de datos

In [2]:
!wget https://raw.githubusercontent.com/MarkzDG/Metodos_estadisticos/main/Stepwise/datossoja.txt
datos = pd.read_table("datossoja.txt", sep = " ", header=0)
datos.head()

--2023-08-21 00:57:44--  https://raw.githubusercontent.com/MarkzDG/Metodos_estadisticos/main/Stepwise/datossoja.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11131 (11K) [text/plain]
Saving to: ‘datossoja.txt.6’

datossoja.txt.6     100%[===================>]  10.87K  --.-KB/s    in 0s      

2023-08-21 00:57:44 (77.6 MB/s) - ‘datossoja.txt.6’ saved [11131/11131]



,caso,rendimiento,pp,temp,fertP,fertZ
0,1,24.0,135.75,22.208333,15.22500,15.654961
1,2,21.6,134.21,25.097222,17.41775,12.613341
2,3,34.7,141.85,21.548611,17.41775,12.431216
3,4,33.4,139.98,20.791667,20.15550,10.678874
4,5,36.2,141.47,22.451389,20.15550,20.008346


In [3]:
#arreglo la tabla
datos.set_index("caso", inplace = True)
datos.columns = ["x1", "x2", "x3", "x4", "x5"] #por comodidad para futuros pasos le asignamos nombres genericos
datos.head()

,x1,x2,x3,x4,x5
caso,,,,,
1,24.0,135.75,22.208333,15.22500,15.654961
2,21.6,134.21,25.097222,17.41775,12.613341
3,34.7,141.85,21.548611,17.41775,12.431216
4,33.4,139.98,20.791667,20.15550,10.678874
5,36.2,141.47,22.451389,20.15550,20.008346


## Aplicación del Método

In [17]:
# Regresión Múltiple
X = datos[["x2", "x3", "x4", "x5"]] #Variables Predictoras
y = datos["x1"] # Predictando
reg_mult = LinearRegression()
reg_mult.fit(X, y)
print("Regresión Múltiple:")
print("Ordenada al origen:", reg_mult.intercept_)
print("Coeficientes:", reg_mult.coef_)
print("R-cuadrado:", reg_mult.score(X, y))

Regresión Múltiple:
Ordenada al origen: -76.86862378206067
Coeficientes: [ 0.86354726 -0.49093351 -0.13266775  0.06013764]
R-cuadrado: 0.748961977944917


Lo siguiente es verificar si podemos mejorar el R^2 mediante el enfoque del modelo aditivo, observando cómo el resultado evoluciona al agregar variables predictoras gradualmente.

Es esencial comprender que realizar una regresión lineal con todas las variables disponibles no garantiza que el modelo resultante sea el óptimo. El método se centra en encontrar el conjunto más reducido de variables predictoras que cumplan con los criterios establecidos:

1.   Buscamos minimizar el AIC.
2.   Buscamos maximizar el valor del R^2.

Este proceso implica seleccionar cuidadosamente las variables predictoras que contribuyan a mejorar tanto el ajuste como la simplicidad del modelo, de acuerdo con los criterios mencionados.

In [18]:
# Método Stepwise Forward
variables_seleccionadas = []
variables_restantes = list(X.columns)
mejor_adj_r2 = -float("inf")
mejor_aic = float("inf")
num_variables = len(X.columns)

# Paso 1: Modelo con solo la ordenada al origen
reg_nulo = LinearRegression().fit(np.ones((len(X), 1)), y)
rss_nulo = np.sum((y - reg_nulo.predict(np.ones((len(X), 1)))) ** 2)
aic_nulo = len(y) * np.log(rss_nulo / len(y)) + 2  # AIC = n * log(RSS/n) + 2 * p (p = 1)
adj_r2_nulo = 1 - (1 - reg_nulo.score(np.ones((len(X), 1)), y)) * ((len(y) - 1) / (len(y) - 1))

print("Paso 1: Modelo con ordenada al origen")
print(f"R-cuadrado ajustado: {adj_r2_nulo}")
print(f"AIC: {aic_nulo}")
print("------")

for _ in range(num_variables):
    mejor_r2 = -float("inf")
    mejor_variable = None
    mejor_temp_adj_r2 = -float("inf")
    mejor_temp_aic = float("inf")

    for variable in variables_restantes:
        variables_actuales = variables_seleccionadas + [variable]
        X_subset = X[variables_actuales]
        reg_temp = LinearRegression()
        reg_temp.fit(X_subset, y)
        r2 = reg_temp.score(X_subset, y)

        if r2 > mejor_r2:
            mejor_r2 = r2
            mejor_variable = variable

            # Calcular el R-cuadrado ajustado temporal
            n = len(y)
            p = len(variables_actuales)
            temp_adj_r2 = 1 - (1 - mejor_r2) * ((n - 1) / (n - p - 1))

            if temp_adj_r2 > mejor_temp_adj_r2:
                mejor_temp_adj_r2 = temp_adj_r2

            # Calcular el AIC temporal
            rss = np.sum((y - reg_temp.predict(X_subset)) ** 2)
            temp_aic = n * np.log(rss / n) + 2 * p

            if temp_aic < mejor_temp_aic:
                mejor_temp_aic = temp_aic

    variables_seleccionadas.append(mejor_variable)
    variables_restantes.remove(mejor_variable)

    X_seleccionado = X[variables_seleccionadas]
    reg_seleccionado = LinearRegression()
    reg_seleccionado.fit(X_seleccionado, y)
    adj_r2 = mejor_temp_adj_r2
    aic = mejor_temp_aic

    if adj_r2 > mejor_adj_r2:
        mejor_adj_r2 = adj_r2
        mejores_variables_seleccionadas = variables_seleccionadas.copy()

    print(f"Paso {_ + 2}: Variable agregada: {mejor_variable}")
    print(f"R-cuadrado ajustado: {adj_r2}")
    print(f"AIC: {aic}")
    print("------")

print("Método Stepwise Forward:")
print("Variables seleccionadas:", mejores_variables_seleccionadas)
print("Mejor R-cuadrado ajustado:", mejor_adj_r2)
print("Coeficientes:", reg_seleccionado.coef_)

Paso 1: Modelo con ordenada al origen
R-cuadrado ajustado: 0.0
AIC: 763.3314306785277
------
Paso 2: Variable agregada: x2
R-cuadrado ajustado: 0.7054257016396288
AIC: 531.3230882717061
------
Paso 3: Variable agregada: x3
R-cuadrado ajustado: 0.7399558474315866
AIC: 508.7451943657751
------
Paso 4: Variable agregada: x4
R-cuadrado ajustado: 0.7443858862936059
AIC: 506.47883237221134
------
Paso 5: Variable agregada: x5
R-cuadrado ajustado: 0.7435046296393717
AIC: 508.10491643157997
------
Método Stepwise Forward:
Variables seleccionadas: ['x2', 'x3', 'x4']
Mejor R-cuadrado ajustado: 0.7443858862936059
Coeficientes: [ 0.86354726 -0.49093351 -0.13266775  0.06013764]


Con esto obtuvimos que el modelo más óptimo para establecer una regresión múltiple es aquel donde x1 es función de la suma de x2, x3 y x4, por lo que el rendimiento de la soja puede ser explicado de manera optima mediante la precipitación (pp), temperatura y el efecto del fertilizante P. La ecuación en cuestión estaría dada por
$$ x_1 = 0.86  - 0.49 \cdot x_2 - 0.13 \cdot x_3 + 0.06 \cdot x_4 $$